<a href="https://colab.research.google.com/github/trtrgfh/GlVYfAbQjtuwUHlZ/blob/main/Potential_Talents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score, f1_score, make_scorer
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("/content/potential-talents.csv")
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [3]:
# Get the unique items used in one-hot encoding columns
location = df["location"].unique()
location     

array(['Houston, Texas', 'Kanada', 'Raleigh-Durham, North Carolina Area',
       'Denton, Texas', 'İzmir, Türkiye', 'Greater New York City Area',
       'San Francisco Bay Area', 'Greater Philadelphia Area',
       'Lake Forest, California', 'Houston, Texas Area',
       'Atlanta, Georgia', 'Chicago, Illinois', 'Austin, Texas Area',
       'Jackson, Mississippi Area', 'Greater Grand Rapids, Michigan Area',
       'Virginia Beach, Virginia', 'Monroe, Louisiana Area',
       'Greater Boston Area', 'San Jose, California',
       'New York, New York', 'Dallas/Fort Worth Area',
       'Amerika Birleşik Devletleri', 'Baton Rouge, Louisiana Area',
       'Myrtle Beach, South Carolina Area', 'Chattanooga, Tennessee Area',
       'Los Angeles, California', 'Highland, California',
       'Gaithersburg, Maryland', 'Baltimore, Maryland',
       'Milpitas, California', 'Greater Atlanta Area',
       'Greater Chicago Area', 'Torrance, California',
       'Long Beach, California', 'Bridgewater, Massa

In [4]:
df['contain_key_word'] = df['job_title'].str.contains('HR|Human Resources', case=False)
df['contain_key_word'] = df['contain_key_word'].astype(int)

In [5]:
df['advanced_degree'] = df['job_title'].str.contains('college|university', case=False)
df['advanced_degree'] = df['advanced_degree'].astype(int)

In [6]:
df

,id,job_title,location,connection,fit,contain_key_word,advanced_degree
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN,1,1
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN,0,0
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,1,0
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,0,0
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,0,1
...,...,...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN,1,0
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN,1,0
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN,0,0
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN,0,0


In [7]:
df['connection'] = df['connection'].str.replace('500\+', '500')
df['connection'] = pd.to_numeric(df['connection'], errors='coerce')


<ipython-input-7-883984c0af28>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['connection'] = df['connection'].str.replace('500\+', '500')


In [8]:
df

,id,job_title,location,connection,fit,contain_key_word,advanced_degree
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN,1,1
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500,NaN,0,0
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,1,0
3,4,People Development Coordinator at Ryan,"Denton, Texas",500,NaN,0,0
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500,NaN,0,1
...,...,...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN,1,0
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500,NaN,1,0
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN,0,0
102,103,Always set them up for Success,Greater Los Angeles Area,500,NaN,0,0


In [9]:
df["connection"] = pd.cut(df["connection"], 
                  bins=[0, 100, 200, 300, 400, 500], 
                  labels=["very_low", "low", "medium", "high", "perfect"],
                  include_lowest=True)

In [10]:
df

,id,job_title,location,connection,fit,contain_key_word,advanced_degree
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",very_low,NaN,1,1
1,2,Native English Teacher at EPIK (English Progra...,Kanada,perfect,NaN,0,0
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",very_low,NaN,1,0
3,4,People Development Coordinator at Ryan,"Denton, Texas",perfect,NaN,0,0
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",perfect,NaN,0,1
...,...,...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",low,NaN,1,0
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",perfect,NaN,1,0
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,very_low,NaN,0,0
102,103,Always set them up for Success,Greater Los Angeles Area,perfect,NaN,0,0
